# Math Agent Example (FIXED VERSION)
**Note: If you continue to see authentication errors after updating .env, please Restart the Kernel.**

This notebook uses the modern LangChain 1.x and LangGraph pattern.

In [ ]:
%pip install -qU python-dotenv langchain langchain-openai langgraph langchain-core

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langgraph.prebuilt import create_react_agent

# 1. Load environment variables from .env file
# We use override=True to ensure the latest key in the file is used
load_dotenv(override=True)

if not os.getenv("OPENAI_API_KEY"):
    print("Warning: OPENAI_API_KEY not found in .env file or environment.")

# 2. Define a simple calculator tool
def calculator(expression: str) -> str:
    """Evaluates a mathematical expression and returns the result."""
    try:
        # Note: eval is restricted for basic security here
        result = eval(expression, {"__builtins__": None}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

# 3. Create the tool
calc_tool = Tool(
    name="Calculator",
    func=calculator,
    description="Useful for performing mathematical calculations. Input should be a valid Python mathematical expression like '25 * 17'."
)

# 4. Initialize the LLM
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

tools = [calc_tool]

# 5. Create the agent using the modern LangGraph prebuilt helper
agent = create_react_agent(llm, tools)

# 6. Test the agent
try:
    print("Invoking agent...")
    inputs = {"messages": [("user", "What is 25 * 17?")]}
    result = agent.invoke(inputs)
    
    # The response is the last message in the list
    final_response = result["messages"][-1].content
    print(f"Agent Output: {final_response}")
except Exception as e:
    print(f"An error occurred: {e}")